In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable 

In [2]:
batch_size = 64
train_dataset = datasets.MNIST(root = './data/',
                               train = True,
                               transform = transforms.ToTensor(),
                               download = True)

test_dataset = datasets.MNIST(root = './data/',
                              train = False,
                              transform = transforms.ToTensor(),
                              download = True)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [3]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

In [14]:
class InceptionA(nn.Module):

  def __init__(self, channels_in):
    super(InceptionA, self).__init__()
    self.branch1x1 = nn.Conv2d(channels_in, 16, kernel_size = 1)

    self.branch5x5_1 = nn.Conv2d(channels_in, 16, kernel_size = 1)
    self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size = 5, padding = 2)

    self.branch3x3_1 = nn.Conv2d(channels_in, 16, kernel_size = 1)
    self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size = 3, padding = 1)
    self.branch3x3_3 = nn.Conv2d(24, 24, kernel_size = 3, padding = 1)

    self.branch_pool = nn.Conv2d(channels_in, 24, kernel_size = 1)

  def forward(self, x):
    branch1 = self.branch1x1(x)

    branch5 = self.branch5x5_1(x)
    branch5 = self.branch5x5_2(branch5)

    branch3 = self.branch3x3_1(x)
    branch3 = self.branch3x3_2(branch3)
    branch3 = self.branch3x3_3(branch3)

    branch_pool = F.avg_pool2d(x, kernel_size = 3, padding = 1, stride = 1)
    branch_pool = self.branch_pool(branch_pool)

    output = [branch1, branch5, branch3, branch_pool]
    return torch.cat(output, 1)




In [15]:
class FinalInception(nn.Module):

  def __init__(self):
    super(FinalInception, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
    self.conv2 = nn.Conv2d(88, 20, kernel_size = 5)

    self.incept1 = InceptionA(channels_in = 10)
    self.incept2 = InceptionA(channels_in = 20)

    self.maxpool = nn.MaxPool2d(2)
    self.fc = nn.Linear(1408, 10)

  def forward(self, x):
    in_size = x.size(0)  #batch_dims
    x = F.relu(self.maxpool(self.conv1(x)))
    x = self.incept1(x)
    x = F.relu(self.maxpool(self.conv2(x)))
    x = self.incept2(x)

    x = x.view(in_size, -1)
    x = self.fc(x)
    return F.log_softmax(x)  

In [16]:
model = FinalInception()

optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [19]:
def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data = Variable(data)
    target = Variable(target)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 10 == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
    


In [21]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 0.050857
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.190557
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.201703
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.060699
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.192593
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.202767
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.254399
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.161281
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.231411
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.149099
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.129429
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.163136
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.234667
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.208653
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.209063
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.256103
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.074297
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.138103
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.134010
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.025805
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0831, Accuracy: 9747/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.165445
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.055947
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.070513
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.226395
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.103977
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.226333
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.075059
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.032122
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.102639
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.076079
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.099774
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.047514
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.110387
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.050999
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.185343
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.041338
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.117100
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.103892
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.069249
Train Epoch: 2 [12